In [17]:
import pandas as pd
import cv2
import os
import csv
from datetime import datetime
from io import StringIO

# Cargar los datos del archivo CSV
file_path = 'E:/Master IA/Practicas Empresa/Imagen/taza_dataset_angle/Material de Videos y Angulos/figura4.csv'

# Leer el archivo CSV sin modificaciones de separadores ni decimales
data = pd.read_csv(file_path)

# Verificar y seleccionar columnas relevantes
if 'time' in data.columns:
    # Convertir el tiempo ISO 8601 a milisegundos
    def iso_to_milliseconds(iso_time):
        dt = datetime.strptime(iso_time.replace(',', '.'), "%Y-%m-%dT%H:%M:%S.%fZ")
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + dt.microsecond / 1000

    # Crear columna de tiempo en milisegundos a partir del campo `time`
    data['time_ms'] = data['time'].apply(iso_to_milliseconds)
else:
    print("Error: No se encontró una columna 'time' en el archivo CSV.")
    exit()

# Seleccionar las columnas necesarias (suponiendo que son `yaw`, `pitch`, `roll`)
if {'yaw', 'pitch', 'roll'}.issubset(data.columns):
    data = data[['time_ms', 'yaw', 'pitch', 'roll']]
else:
    print("Error: Las columnas 'yaw', 'pitch', 'roll' no se encontraron en el archivo CSV.")
    exit()

# Parámetros de archivos y carpeta de salida
output_csv = 'angulo_frames_figuras.csv'
video_path = 'E:/Master IA/Practicas Empresa/Imagen/taza_dataset_angle/Material de Videos y Angulos/figura4.mp4'
output_folder = 'E:/Master IA/Practicas Empresa/Imagen/taza_dataset_angle/Feature_img/capturas'

# Crear la carpeta para las capturas si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Inicializar lista de resultados
resultados = []

# Abrir el video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: no se pudo abrir el video {video_path}")
    exit()

# Obtener FPS y total de frames del video
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    print("Error: No se pudo obtener FPS del video")
    exit()

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {total_frames}, FPS: {fps}")

# Obtener el tiempo inicial del CSV para alinear correctamente los tiempos del video
start_time_ms = data['time_ms'].iloc[0]

# Procesar cada tercer frame
for frame_number in range(0, total_frames, 3): 
    # Calcular el tiempo en milisegundos del frame actual
    current_time_ms = start_time_ms + (frame_number / fps) * 1000

    # Encontrar el registro más cercano en el CSV
    closest_index = (data['time_ms'] - current_time_ms).abs().idxmin()
    closest_record = data.iloc[closest_index]

    # Obtener yaw, pitch y roll del registro más cercano
    yaw_value = closest_record['yaw']
    pitch_value = closest_record['pitch']
    roll_value = closest_record['roll']

    # Mover el video al frame correspondiente
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Leer el frame
    ret, frame = cap.read()
    if ret:
        # Guardar la imagen en la carpeta de capturas
        image_name = f'figura4_frame_{int(frame_number/3)}.png'
        cv2.imwrite(os.path.join(output_folder, image_name), frame)

        # Agregar los resultados a la lista
        resultados.append([image_name, yaw_value, pitch_value, roll_value])
    else:
        print(f'Error al capturar el frame en el frame number {frame_number}')

# Convertir los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados, columns=['image_name', 'yaw', 'pitch', 'roll'])

# Convertir los valores a formato string con comas como separadores decimales
df_resultados['yaw'] = df_resultados['yaw'].apply(lambda x: f"{x:.6f}".replace('.', ','))
df_resultados['pitch'] = df_resultados['pitch'].apply(lambda x: f"{x:.6f}".replace('.', ','))
df_resultados['roll'] = df_resultados['roll'].apply(lambda x: f"{x:.6f}".replace('.', ','))

# Guardar los resultados en un archivo CSV, añadiendo al final si el archivo ya existe
if not os.path.isfile(output_csv):
    # Si el archivo no existe, lo creamos y escribimos los encabezados
    df_resultados.to_csv(output_csv, index=False, sep=';', quoting=csv.QUOTE_NONNUMERIC)
else:
    # Si el archivo ya existe, añadimos al final sin escribir los encabezados
    df_resultados.to_csv(output_csv, mode='a', header=False, index=False, sep=';', quoting=csv.QUOTE_NONNUMERIC)

# Liberar el video
cap.release()
print(f"Proceso completo. Los resultados se han guardado en {output_csv}")


Total frames in video: 446, FPS: 27.020750239814205
Proceso completo. Los resultados se han guardado en angulo_frames_figuras.csv
